This is playing with pytorch framework for EHR modeling. In general, a patient's health record can be represented as a sequence of visits. Each visit has certain features, and can be represented as a list of medical codes.

For simplicity, we are starting with the data structure that a patient's health record is a list of list, following the line of work from Jimeng Sun's lab. We will use codes from Ed Choi to manipulate the data. 

The core model is an RNN , either LSTM, GRU or Vanilla RNN.

# todos:
* None for now

In [1]:
%matplotlib inline
from __future__ import print_function, division
from io import open
import string
import re
import random
import sklearn 
from sklearn.metrics import roc_auc_score
import plotly.plotly as py 
import plotly.graph_objs as go
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.utils.data as Data

use_cuda = torch.cuda.is_available()
torch.cuda.set_device(0)

import sys, random
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

from torchviz import make_dot, make_dot_from_trace
random.seed(9001)
# for windows only    
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
#os.environ["CUDA_DEVICE_ORDER"]='PCI_BUS_ID'
#os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [2]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=1
use_cuda

True

In [3]:
train_s = pickle.load(open('pdata_3hosp/h143_train', 'rb'), encoding='bytes')
valid_sl = pickle.load(open('pdata_3hosp/h143_valid', 'rb'), encoding='bytes')
test_sl =  pickle.load(open('pdata_3hosp/h143_test', 'rb'), encoding='bytes')
train_size=len(train_s)
#test_sh_143 = pickle.load(open('pdata_3hosp/h143_test_sh', 'rb'), encoding='bytes')
#test_l_143 = pickle.load(open('pdata_3hosp/h143_test_lng', 'rb'), encoding='bytes')
print (train_size,len(valid_sl),len(test_sl))

29912 4272 8545


In [4]:
from collections import OrderedDict
def load_params(model):
    params = OrderedDict()
    weights = np.load(model)
    for k,v in weights.iteritems():
        params[k] = v
    return params

param= load_params("/data/projects/cerner_dev/cl2_hosp_retain_out/h50_cl2_all_hosp_143_cl2m.out.0.npz")
emb_pretrain = param['W_emb']


In [5]:
class EHR_RNN(nn.Module):
    def __init__(self, input_size,embed_dim, hidden_size, n_layers=1,dropout_r=0.1,cell_type='LSTM',bi=False , preTrainEmb=''):
        super(EHR_RNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb=preTrainEmb
        if bi: self.bi=2 
        else: self.bi=1
              
        if len(self.preTrainEmb)>0:
            emb_t= torch.FloatTensor(np.asmatrix(self.preTrainEmb))
            self.embed= nn.Embedding.from_pretrained(emb_t)#,freeze=False)  
        else:
            self.embed= nn.Embedding(input_size, self.embed_dim,padding_idx=0)
        
        if self.cell_type == "GRU":
            cell = nn.GRU
        elif self.cell_type == "RNN":
            cell = nn.RNN
        elif self.cell_type == "LSTM":
            cell = nn.LSTM
        else:
            raise NotImplementedError
      
        self.rnn_c = cell(self.embed_dim, hidden_size,num_layers=n_layers, dropout= dropout_r , bidirectional=bi , batch_first=True )
        self.out = nn.Linear(self.hidden_size*self.bi,1)
        self.sigmoid = nn.Sigmoid()

        
    def EmbedPatient_MB(self, input): # x is a ehr_seq_tensor
        
        mb=[]
        lbt=[]
        seq_l=[]
        lp= len(max(input, key=lambda xmb: len(xmb[1]))[1])
        #print (lp)
        llv=0
        for x in input:
            lv= len(max(x[1], key=lambda xmb: len(xmb)))
            if llv< lv:
                llv=lv
        #print (llv)
        for pt in input:
            label, ehr_seq_l = pt
            lpx=len(ehr_seq_l)
            seq_l.append(lpx)
            label_tensor = Variable(torch.FloatTensor([[float(label)]]))
            if use_cuda:
                label_tensor = label_tensor.cuda()
            lbt.append(label_tensor)
            ml=(len(max(ehr_seq_l, key=len)))
            ehr_seq_tl=[]
            for ehr_seq in ehr_seq_l: 
                pd=(0, llv-len(ehr_seq))
                result = F.pad(torch.from_numpy(np.asarray(ehr_seq,dtype=int)).type(torch.cuda.LongTensor),pd,"constant", 0)
                if use_cuda:
                    result.cuda()
                ehr_seq_tl.append(result)
            ehr_seq_t= Variable(torch.stack(ehr_seq_tl,0))     
            lpp= lp-lpx
            zp= nn.ZeroPad2d((0,0,0,lpp))
            ehr_seq_t= zp(ehr_seq_t)
            mb.append(ehr_seq_t)
                
        mb_t= Variable(torch.stack(mb,0)) 
        if use_cuda:
            mb_t.cuda()
        embedded = self.embed(mb_t)
        embedded = torch.sum(embedded, dim=2) 
        lbt_t= Variable(torch.stack(lbt,0)) 
        return embedded, lbt_t,seq_l
    
    
    def forward(self, input):
        
        x_in , lt ,x_lens = self.EmbedPatient_MB(input)
        x_inp = nn.utils.rnn.pack_padded_sequence(x_in,x_lens,batch_first=True)
        output, hidden = self.rnn_c(x_inp) 
        if self.cell_type == "LSTM":
            hidden=hidden[0]
        if self.bi==2:
            output = self.sigmoid(self.out(torch.cat((hidden[-2],hidden[-1]),1)))
        else:
            output = self.sigmoid(self.out(hidden[-1]))
        return output.squeeze(), lt.squeeze()



In [6]:
emb_pretrain=''                       
model = EHR_RNN(input_size=15816, hidden_size=128 ,embed_dim=128, dropout_r=0, cell_type='GRU',bi=False , n_layers=1, preTrainEmb=emb_pretrain)
#bmodel_pth = './best_model_GRUPre.pth'
#bmodel_st = './best_model_GRUPre.st'
if use_cuda:
    model = model.cuda()

In [7]:
def train (mini_batch, criterion, optimizer):  
    
    model.zero_grad()
    output , label_tensor = model(mini_batch,)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
   
    return output, loss.item()

In [8]:
# training all samples in random order
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)





In [9]:
def run_model_train(dataset,batch_size,optimizer):#learning_rate = 0.01, l2=1e-05,epsl=1e-06 ):
    

    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    # Keep track of losses for plotting
    current_loss = 0
    all_losses = []
    print_every = 10#int(batch_size/2)
    plot_every = 5
    iter=0
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    #print('number of Batches',n_batches)
    start = time.time()

    for index in random.sample(range(n_batches), n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, loss = train(batch, criterion = nn.BCELoss(), optimizer = optimizer)
            current_loss += loss
            iter +=1
            # Print iter number, loss, name and guess
            #if iter % print_every == 0:
               #print('%d %d%% (%s) %.4f ' % ( iter, iter/ n_batches * 100, timeSince(start), loss))

            # Add current loss avg to list of losses
            if iter % plot_every == 0:
                all_losses.append(current_loss / plot_every)
                current_loss = 0
                
    return current_loss,all_losses


In [10]:
def calculate_auc(test_model, dataset, batch_size=200):

    dataset.sort(key=lambda pt:len(pt[1]),reverse=True) 
    n_batches = int(np.ceil(int(len(dataset)) / int(batch_size)))
    labelVec =[]
    y_hat= []
    
    for index in range(n_batches):
            batch = dataset[index*batch_size:(index+1)*batch_size]
            output, label_t = test_model(batch)
            y_hat.extend(output.cpu().data.view(-1).numpy())
            labelVec.extend(label_t.cpu().data.view(-1).numpy())
    auc = roc_auc_score(labelVec, y_hat)
    
    return auc

In [ ]:
te_results=[]
te_t_s=[]
te_d_s=[]
for percent_t in [0.01,0.02,0.04,0.06,0.1,0.2,0.4,0.6,1]:#,2,4,6,10,15,25,50,75,100]
    emb_pretrain=''                       
    model = EHR_RNN(input_size=15816, hidden_size=128 ,embed_dim=128, dropout_r=0, cell_type='GRU',bi=False , n_layers=1, preTrainEmb=emb_pretrain)
    #bmodel_pth = './best_model_GRUPre.pth'
    #bmodel_st = './best_model_GRUPre.st'
    if use_cuda:
        model = model.cuda()
    nTrain = int(percent_t * train_size)
    te_res=[percent_t,nTrain]
    random.shuffle(train_s)
    train_sl = train_s[:nTrain]
    epochs=100
    batch_size=100
    current_loss_l=[]
    all_losses_l=[]
    train_auc_allep =[]
    valid_auc_allep =[]
    test_auc_allep=[]
    #tests_auc_allep=[]
    #testl_auc_allep=[]
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
    learning_rate= 0.01
    l2= 1e-05
    epsl=1e-06
    #optimizer = optim.SGD(model.parameters(), lr=learning_rate)#, weight_decay=l2)##lr 0.1
    #optimizer = optim.Adadelta(model.parameters(), lr=learning_rate, weight_decay=l2)## slow performing Lr 1 is the default and make it converge faster
    #optimizer = optim.ASGD(model.parameters(), lr=learning_rate, weight_decay=l2 )##lr 0.1
    #optimizer = optim.SparseAdam (model.parameters(),lr=learning_rate) #'''lr=learning_rate,''' 
    optimizer = optim.Adamax (model.parameters(),lr=learning_rate, weight_decay=l2,eps=epsl) #'''lr=learning_rate,''' 
    #optimizer = optim.Adamax(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=l2 ,eps=epsl) ### Beta defaults (0.9, 0.999)
    #optimizer = optim.RMSprop (model.parameters(),lr=learning_rate, weight_decay=l2 ,eps=epsl)#lr 0.001
    #optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2,eps=epsl )
    #optimizer = optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=l2 )
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',patience=3, verbose= True, threshold=0.0001, threshold_mode='abs')
    for ep in range(epochs):
        start = time.time()
        current_loss_la,all_losses_la = run_model_train(train_sl,batch_size,optimizer)
        train_time = timeSince(start)
        eval_start = time.time()
        train_auc = calculate_auc(model,train_sl,batch_size)
        test_auc = calculate_auc(model,test_sl,batch_size)
        valid_auc = calculate_auc(model,valid_sl,batch_size)
        scheduler.step(valid_auc)
        eval_time = timeSince(eval_start)
        all_losses_l.append (all_losses_la)
        avg_loss = np.mean(all_losses_la)
        train_auc_allep.append(train_auc)
        valid_auc_allep.append(valid_auc)
        test_auc_allep.append(test_auc)
        #testl_auc_allep.append(test_l_auc)
        #tests_auc_allep.append(test_sh_auc)
        current_loss_l.append(current_loss_la)
        print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc," Avg Loss: ", avg_loss, ',Lr ',optimizer.param_groups[0]['lr'], ' Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
        #print ("Epoch ", ep," Train_auc :", train_auc, " , Valid_auc : ", valid_auc, " ,& Test_auc : " , test_auc,"less than 5 visits history :",test_sh_auc," for more :",test_l_auc ," Avg Loss: ", avg_loss, 'Train Time (%s) Eval Time (%s)'%(train_time,eval_time) )
        if valid_auc > bestValidAuc: 
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            bestTestAuc = test_auc
            train_err= round(1-train_auc,3)
            dev_err=round(1-valid_auc,3)
            #best_model = model
            #torch.save(best_model, bmodel_pth)
            #torch.save(best_model.state_dict(), bmodel_st)
        if ep - bestValidEpoch >6: break
    te_t_s.append(train_err)
    te_d_s.append(dev_err)
    te_res.append(train_auc_allep)
    te_res.append(valid_auc_allep)
    te_res.append(test_auc_allep)
    print ('bestValidAuc %f has a TestAuc of %f at epoch %d ' % (bestValidAuc, bestTestAuc, bestValidEpoch))
    te_results.append(te_res)
    
    #torch.save(best_model, bmodel_pth)



/home/lgindybekhet/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning:

Mean of empty slice.

/home/lgindybekhet/.local/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning:

invalid value encountered in double_scalars



Epoch  0  Train_auc : 0.7288954635108481  , Valid_auc :  0.5836979334862222  ,& Test_auc :  0.582973977048154  Avg Loss:  nan ,Lr  0.01  Train Time (0m 0s) Eval Time (0m 16s)
Epoch  1  Train_auc : 0.8813609467455621  , Valid_auc :  0.6030423690239862  ,& Test_auc :  0.6029074069591972  Avg Loss:  nan ,Lr  0.01  Train Time (0m 0s) Eval Time (0m 16s)
Epoch  2  Train_auc : 0.9565088757396449  , Valid_auc :  0.6155587138753513  ,& Test_auc :  0.6121844284351827  Avg Loss:  nan ,Lr  0.01  Train Time (0m 0s) Eval Time (0m 16s)
Epoch  3  Train_auc : 0.9885601577909271  , Valid_auc :  0.6359987257436395  ,& Test_auc :  0.6304554417695967  Avg Loss:  nan ,Lr  0.01  Train Time (0m 0s) Eval Time (0m 16s)
Epoch  4  Train_auc : 0.995266272189349  , Valid_auc :  0.6465446724663908  ,& Test_auc :  0.6404866423345952  Avg Loss:  nan ,Lr  0.01  Train Time (0m 0s) Eval Time (0m 17s)


In [ ]:
# plot trainiing versus dev error
import plotly.plotly as py 
import plotly.graph_objs as go
py.sign_in('LailaRasmy','mzNHzVvwYjcZwBDZx3B7')

x_l=[0.01,0.02,0.04,0.06,0.1,0.2,0.4,0.6,1]
train_err_fg= go.Scatter(x=x_l, y=te_t_s, name='train_err')
dev_err_fg= go.Scatter(x=x_l, y=te_d_s, name='dev_err')

train_err_max = max(te_t_s)
dev_err_min = min(te_d_s)
test_max = max(test_auc_allep)
data = [train_err_fg,dev_err_fg]
layout = go.Layout(xaxis=dict(dtick=0.1))
#layout.update(dict(annotations=[go.Annotation(text="Max train error ", x=x_l[te_t_s.index(train_err_max)], y=train_err_max)]))
layout.update(dict(annotations=[go.Annotation(text="Min Dev error ", x=x_l[te_d_s.index(dev_err_min)], y=dev_err_min)]))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Training Dev error plot')
#url = py.plot(data, filename='some-data')  # gen. online plot
#py.image.save_as(data, 'some-data.png') 